In [2]:
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as img # mpimg 用于读取图片
import numpy as np
import csv
import os
import cv2
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata, nn
import sys

In [3]:
g = os.walk("G:/桌面/chineseminst/train_data/")
for path,d,filelist in g:  
    pass
train_data=[]
for i in range(len(filelist)):
    img_temp=img.imread(r'G:/桌面/chineseminst/train_data/%s'%filelist[i])
    r,g,b = [img_temp[:,:,i] for i in range(3)]
    img_gray = r*0.299+g*0.587+b*0.114
    img_gray = (img_gray - img_gray.min()) * (1 / (img_gray.max() - img_gray.min()))
    sp=filelist[i].split('_')
    label=sp[3].split('.')
    temp=[img_gray,int(label[0])]
    train_data.append(temp)
    

In [4]:
g = os.walk("G:/桌面/chineseminst/test_data/")
for path,d,filelist in g:  
    pass
test_data=[]
for i in range(len(filelist)):
    img_temp=img.imread(r'G:/桌面/chineseminst/test_data/%s'%filelist[i])
    r,g,b = [img_temp[:,:,i] for i in range(3)]
    img_gray = r*0.299+g*0.587+b*0.114
    img_gray = (img_gray - img_gray.min()) * (1 / (img_gray.max() - img_gray.min()))
    sp=filelist[i].split('_')
    label=sp[3].split('.')
    temp=[img_gray,int(label[0])]
    test_data.append(temp)
    

In [17]:
net = nn.Sequential()
net.add(nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=16, kernel_size=5, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        # Dense会默认将(批量大小, 通道, 高, 宽)形状的输入转换成
        # (批量大小, 通道 * 高 * 宽)形状的输入
        nn.Dense(120, activation='sigmoid'),
        nn.Dense(84, activation='sigmoid'),
        nn.Dense(15))

In [19]:
train_iter=ImgLoader(train_data,batch_size)
net.initialize()
for x,y in train_iter:
    print(np.array(x).shape)
    break
x=nd.array(x)
for layer in net:
    x = layer(x)
    print(layer.name, 'output shape:\t', x.shape)

(200, 1, 64, 64)
conv2 output shape:	 (200, 6, 60, 60)
pool2 output shape:	 (200, 6, 30, 30)
conv3 output shape:	 (200, 16, 26, 26)
pool3 output shape:	 (200, 16, 13, 13)
dense3 output shape:	 (200, 120)
dense4 output shape:	 (200, 84)
dense5 output shape:	 (200, 15)


In [8]:
batch_size=200

In [18]:
def ImgLoader(data, batch_size, shuffle=True):
    if shuffle:
        sf_idx = list(range(len(data)))
        np.random.shuffle(sf_idx)
        data = np.array(data)
        data = data[sf_idx]
    batch_data = []
    batch_label = []
    for item in data:
        batch_data.append([item[0]])
        batch_label.append(int(item[1] - 1))
        if len(batch_data) == batch_size:
            yield batch_data, batch_label
            batch_data = []
            batch_label = []

In [10]:
def evaluate_accuracy(data_iter, net, ctx):
    acc_sum, n = nd.array([0], ctx=ctx), 0
    for X, y in data_iter:
#         # 如果ctx代表GPU及相应的显存，将数据复制到显存上
        X=nd.array(X)
        y=nd.array(y)
        X, y = X.as_in_context(ctx), y.as_in_context(ctx).astype('float32')
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.size
    return acc_sum.asscalar() / n

In [14]:
def train_ch5(net, batch_size, trainer, ctx,
              num_epochs):
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    best_acc = 0
    for epoch in range(num_epochs):
        train_iter = ImgLoader(train_data, batch_size=batch_size)
        test_iter = ImgLoader(test_data, batch_size=batch_size,shuffle=False)
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X=nd.array(X)             
            y=nd.array(y)
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        val_acc = test_acc
        if val_acc > best_acc:
            best_acc = val_acc
            # save model
            net.save_parameters("Chinese_minst.params")
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [22]:
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn
import time
ctx = mx.gpu(0)
lr, num_epochs = 0.9, 30
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 2.7243, train acc 0.085, test acc 0.131, time 2.3 sec
epoch 2, loss 2.2925, train acc 0.271, test acc 0.381, time 2.3 sec
epoch 3, loss 1.8501, train acc 0.411, test acc 0.462, time 2.3 sec
epoch 4, loss 1.2439, train acc 0.608, test acc 0.628, time 2.4 sec
epoch 5, loss 0.7798, train acc 0.755, test acc 0.800, time 2.5 sec
epoch 6, loss 0.5214, train acc 0.830, test acc 0.845, time 2.6 sec
epoch 7, loss 0.3781, train acc 0.879, test acc 0.865, time 2.6 sec
epoch 8, loss 0.2871, train acc 0.914, test acc 0.904, time 2.8 sec
epoch 9, loss 0.5812, train acc 0.859, test acc 0.884, time 2.7 sec
epoch 10, loss 0.2241, train acc 0.938, test acc 0.917, time 2.8 sec
epoch 11, loss 0.1428, train acc 0.966, test acc 0.912, time 2.9 sec
epoch 12, loss 0.1043, train acc 0.979, test acc 0.927, time 2.9 sec
epoch 13, loss 0.0894, train acc 0.982, test acc 0.902, time 3.0 sec
epoch 14, loss 0.0734, train acc 0.986, test acc 0.944, time 3.1 sec
epoch 15, loss 0.4045, t